In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tables as tb

from PIL import Image 
# Nedded for background fit
import numpy.polynomial.polynomial as poly
# needed for spotfinder
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters

In [2]:
from SMFI_Definitions import Image_Background
from SMFI_Definitions import Summed_Image
from SMFI_Definitions import Spot_finder
from SMFI_Definitions import Spot_Area
from SMFI_Definitions import Step_fit
from SMFI_Definitions import Line_fit

class Trajectory():
    def __init__(self):
        name           = ""
        Mean_diff      = 0 
        Chi_Step       = 0
        Chi_Line       = 0
        Step_Time      = 0
        Times          = np.array([])
        Signal         = np.array([])
        Signal_er      = np.array([])
        Background     = np.array([])
        Background_er  = np.array([])

In [12]:
SLIDE_LOC = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/"

SLIDE_PATHS = [os.path.join(SLIDE_LOC, f) for f in os.listdir(SLIDE_LOC) if f.endswith(".h5")] 
SLIDE_NAMES = []
for slide in SLIDE_PATHS:
    hold = slide.split("/")[-1].split(".")[0]
    SLIDE_NAMES.append(hold)

In [16]:
Traject = {}
# should be the shape of the image
Shape = 221
# this is the reaction of the final images it will use as a background 
Frac = 0.1
#The order of the polynominal that will subtract the background
PolyOrder = 2

# these are for the spot finder
# the neighborhood_size is for the nearest spot 
# threshold_sigma is now large above the mean it will look for a peak
# EdgeCut is the boarder it will cut off the image
neighborhood_size = 10
threshold_sigma = 1
EdgeCut = 10  

# the number of frames in the stack
Num_Frames=500


for slide in SLIDE_PATHS:
    SLIDE_NAME = slide.split("/")[-1].split(".")[0]
    print("Startin on "+SLIDE_NAME)
    
    with tb.open_file(slide, mode="r", title="test") as tt:
        Num_Frames = tt.root.ImageStack._v_nchildren
        Frame_Name = []
        for kid in tt.root.ImageStack._v_children:
            Frame_Name.append(kid)

        image_background = Image_Background(tt, Frame_Name, Num_Frames, Shape, Frac, PolyOrder)
        summed_image = Summed_Image(tt, Frame_Name, Num_Frames, Shape, image_background)
        spots = Spot_finder(summed_image, neighborhood_size, threshold_sigma, Shape, EdgeCut)
        spot_area = Spot_Area(tt, Frame_Name, Num_Frames, image_background, spots)
        
        Num_Spots = len(spots)
        print("found "+str(Num_Spots)+" spots!")
        for x in range(0,Num_Spots):
            X = str(int(spots[x][0]))
            Y = str(int(spots[x][1]))
            NAME = "X"+X+"_Y"+Y
            sig = np.array(spot_area[x::Num_Spots])[:,2]
            bkg = np.array(spot_area[x::Num_Spots])[:,3]
            
            SIG = sig
            BAK = bkg
            ww = np.arange(0,Num_Frames)*0.5
            sig = np.add.reduceat(SIG, np.arange(0, len(SIG), 5))
            bak = np.add.reduceat(BAK, np.arange(0, len(BAK), 5))
            www = np.add.reduceat(ww, np.arange(0, len(ww), 5))
            
            
            times    = np.arange(0,len(sig))*2.5
            baerry = np.sqrt(sig)
            naerry = np.sqrt(bak)
            coefsF = poly.polyfit(times,bak, 7)
            ffitF  = poly.polyval(times, coefsF)
            
            
            fitS = (sig-ffitF*25/56)/25
            fitSbk = baerry/25
            fitB = (bak-ffitF)/56
            fitBbk = naerry/25
            tim = times
            Meandiff, chiStep, LT = Step_fit(tim,fitS,fitSbk)
            chiLine     = Line_fit(tim,fitS,fitSbk)
            
            Trj = Trajectory()
            Trj.name = NAME
            Trj.Mean_diff = Meandiff
            Trj.Chi_Step = chiStep
            Trj.Chi_Line = chiLine
            Trj.Step_Time = LT
            Trj.Times = times
            Trj.Signal = fitS
            Trj.Signal_er = fitSbk
            Trj.Background = fitB
            Trj.Background_er = fitBbk
     
    
            Traject[SLIDE_NAME+'/'+NAME] = Trj

np.save('Trajectory.npy',Traject)

Startin on Slide1_Spot3
found 206 spots!
Startin on Slide1_Spot6
found 229 spots!
Startin on Slide1_Spot2
found 220 spots!
Startin on Slide2_Spot4
found 193 spots!
Startin on Slide3_Spot4
found 239 spots!
Startin on Slide2_Spot1
found 212 spots!
Startin on Slide3_Spot1
found 213 spots!
Startin on Slide2_Spot5
found 205 spots!
Startin on Slide3_Spot5
found 207 spots!
Startin on Slide2_Spot2
found 204 spots!
Startin on Slide3_Spot2
found 180 spots!
Startin on Slide2_Spot6
found 222 spots!
Startin on Slide2_Spot3
found 227 spots!
Startin on Slide3_Spot3
found 205 spots!
Startin on Slide1_Spot5
found 210 spots!
Startin on Slide1_Spot1
found 229 spots!
Startin on Slide1_Spot4
found 206 spots!


In [ ]:
###################
### this only runs the above for one spot if you are fo
###################

In [ ]:
tt = tb.open_file("/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/Slide1_Spot1.h5"
                    , mode="r", title="test")

In [ ]:
Num_Frames = tt.root.ImageStack._v_nchildren
Num_Frames

In [ ]:
Frame_Name = []
for kid in tt.root.ImageStack._v_children:
    Frame_Name.append(kid)

In [ ]:
aa = tt.root.ImageStack[Frame_Name[0]].read()
len(aa)

In [ ]:
Shape = len(aa)
Frac = 0.1
PolyOrder = 2

In [ ]:
image_background = Image_Background(tt, Frame_Name, Num_Frames, Shape, Frac, PolyOrder)

In [ ]:
plt.imshow(image_background)

In [ ]:
summed_image = Summed_Image(tt, Frame_Name, Num_Frames, Shape, image_background)

In [ ]:
plt.imshow(summed_image)

In [ ]:
neighborhood_size = 10
threshold_sigma = 1
EdgeCut = 20
spots = Spot_finder(summed_image, neighborhood_size, threshold_sigma, Shape, EdgeCut)
len(spots)

In [ ]:
plt.imshow(summed_image)
plt.scatter(spots[:,0],spots[:,1],color='darkorange',alpha=0.6)

In [ ]:
SLIDE_LOC = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/"

SLIDE_PATHS = [os.path.join(SLIDE_LOC, f) for f in os.listdir(SLIDE_LOC) if f.endswith(".h5")] 
SLIDE_NAMES = []
for slide in SLIDE_PATHS:
    hold = slide.split("/")[-1].split(".")[0]
    SLIDE_NAMES.append(hold)


In [ ]:
ANA_FILE = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana_full-v2.h5"
Shape = 221
Frac = 0.1
    
with tb.open_file(ANA_FILE, mode="w", title="test") as testana:
        
    for slide in SLIDE_PATHS:
        SLIDE_NAME = slide.split("/")[-1].split(".")[0]
        print("Startin on "+SLIDE_NAME)
        GroupName  = "Spot_Info"+"_"+SLIDE_NAME
        GroupInfo  = testana.create_group("/", GroupName)
        GroupName  = "F_trajectory"+"_"+SLIDE_NAME
        GroupF     = testana.create_group("/", GroupName)
        #GroupName  = "Background_Area"+"_"+SLIDE_NAME
        #GroupBkg   = testana.create_group("/", GroupName)

        with tb.open_file(slide, mode="r", title="test") as tt:
            Num_Frames = tt.root.ImageStack._v_nchildren
            Frame_Name = []
            for kid in tt.root.ImageStack._v_children:
                Frame_Name.append(kid)

            mm = Image_Background(Frame_Name, Num_Frames, Shape, Frac)
            testana.create_array(GroupInfo, "Background", mm)
            rr = Summed_Image(Frame_Name, Num_Frames, Shape, mm)
            testana.create_array(GroupInfo, "Summed", mm)
            aa = Spot_finder(rr,10,3,Shape,20)
            testana.create_array(GroupInfo, "AllPairs", aa)

            aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)
            Num_Spots = len(aa)
            print("found "+str(Num_Spots)+" spots!")
            for x in range(0,Num_Spots):
                X = str(int(aa[x][0]))
                Y = str(int(aa[x][1]))
                NAME = "X"+X+"_Y"+Y
                sig = np.array(aaa[x::Num_Spots])[:,2]
                bkg = np.array(aaa[x::Num_Spots])[:,3]
                stuff = np.array([sig,bkg])
                testana.create_array(GroupF, NAME, stuff)
        #testana.flush()
    
    

In [ ]:
qq = 0
for y in range(0,len(ttt)):
#for y in range(2,5):
    spots = []
    for kid in ttry.root[ttt[y]]._v_children:
        spots.append(kid)
        
    for x in range(0,len(spots)):
        SIG = ttry.root[ttt[y]][spots[x]].read()
        BAK = ttry.root[yyy[y]][spots[x]].read()
        Num_Frames=500
        ww = np.arange(0,Num_Frames)*0.5
        sig = np.add.reduceat(SIG, np.arange(0, len(SIG), 5))
        bak = np.add.reduceat(BAK, np.arange(0, len(BAK), 5))
        www = np.add.reduceat(ww, np.arange(0, len(ww), 5))
        times    = np.arange(0,len(sig))*2.5
        baerry = np.sqrt(sig)
        naerry = np.sqrt(bak)
        coefsF = poly.polyfit(times,bak, 3)
        ffitF  = poly.polyval(times, coefsF)
        fitS = (sig-ffitF*25/56)/25
        fitSbk = baerry/25
        fitB = (bak-ffitF)/56
        fitBbk = naerry/25
        tim = times
        Meandiff, chiStep, LT = Step_fit(tim,fitS,sigS)
        chiLine     = Line_fit(tim,fitS,sigS)

In [ ]:
SLIDE_PATHS

In [ ]:
class Trajectory():
    def __init__(self):
        name           = ""
        Mean_diff      = 0 
        Chi_Step       = 0
        Chi_Line       = 0
        Step_Time      = 0
        Signal         = np.array([])
        Signal_er      = np.array([])
        Background     = np.array([])
        Background_er  = np.array([])

In [ ]:
ANA_FILE = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana_full-v2.h5"
Shape = 221
Frac = 0.1
dictt = {}
    
#with tb.open_file(ANA_FILE, mode="w", title="test") as testana:
        
for slide in SLIDE_PATHS:
    SLIDE_NAME = slide.split("/")[-1].split(".")[0]
    print("Startin on "+SLIDE_NAME)
    GroupName  = "Spot_Info"+"_"+SLIDE_NAME
    #GroupInfo  = testana.create_group("/", GroupName)

    GroupName  = "F_Trajectory_Raw"+"_"+SLIDE_NAME
    #GroupFraw  = testana.create_group("/", GroupName)

    GroupName  = "F_Trajectory_Fit"+"_"+SLIDE_NAME
    #GroupFfit  = testana.create_group("/", GroupName)


    with tb.open_file(slide, mode="r", title="test") as tt:
        Num_Frames = tt.root.ImageStack._v_nchildren
        Frame_Name = []
        for kid in tt.root.ImageStack._v_children:
            Frame_Name.append(kid)

        mm = Image_Background(Frame_Name, Num_Frames, Shape, Frac)
        #testana.create_array(GroupInfo, "Background", mm)
        rr = Summed_Image(Frame_Name, Num_Frames, Shape, mm)
        #testana.create_array(GroupInfo, "Summed", mm)
        aa = Spot_finder(rr,10,3,Shape,20)
        #testana.create_array(GroupInfo, "AllPairs", aa)

        aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)
        Num_Spots = len(aa)
        print("found "+str(Num_Spots)+" spots!")
        for x in range(0,Num_Spots):
            X = str(int(aa[x][0]))
            Y = str(int(aa[x][1]))
            NAME = "X"+X+"_Y"+Y
            sig = np.array(aaa[x::Num_Spots])[:,2]
            bkg = np.array(aaa[x::Num_Spots])[:,3]
            stuff = np.array([sig,bkg])
            #testana.create_array(GroupFraw, NAME, stuff)

            SIG = sig
            BAK = bkg
            Num_Frames=500
            ww = np.arange(0,Num_Frames)*0.5
            sig = np.add.reduceat(SIG, np.arange(0, len(SIG), 5))
            bak = np.add.reduceat(BAK, np.arange(0, len(BAK), 5))
            www = np.add.reduceat(ww, np.arange(0, len(ww), 5))
            times    = np.arange(0,len(sig))*2.5
            baerry = np.sqrt(sig)
            naerry = np.sqrt(bak)
            coefsF = poly.polyfit(times,bak, 3)
            ffitF  = poly.polyval(times, coefsF)
            fitS = (sig-ffitF*25/56)/25
            fitSbk = baerry/25
            fitB = (bak-ffitF)/56
            fitBbk = naerry/25
            tim = times
            Meandiff, chiStep, LT = Step_fit(tim,fitS,fitSbk)
            chiLine     = Line_fit(tim,fitS,fitSbk)
            
            Trj = Trajectory()
            Trj.name = NAME
            Trj.Mean_diff = Meandiff
            Trj.Chi_Step = chiStep
            Trj.Chi_Line = chiLine
            Trj.Step_Time = LT
            Trj.Signal = fitS
            Trj.Signal_er = fitSbk
            Trj.Background = fitB
            Trj.Background_er = fitBbk
     
    
            dictt[SLIDE_NAME+'/'+NAME] = Trj
                
                #stuff = np.array([fitS, fitSbk, fitB, fitBbk, Meandiff, chiStep, chiLine, LT])
                #testana.create_array(GroupFfit, NAME, stuff)
                
        #testana.flush()
    
    

In [ ]:
dictt['Slide2_Spot3/X81_Y73'].Mean_diff

In [ ]:
dictt.keys()

In [ ]:
np.save('Trajectory.npy',dictt)

In [ ]:
ANA_FILE = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana_full-v2.h5"
Shape = 221
Frac = 0.1
    
with tb.open_file(ANA_FILE, mode="w", title="test") as testana:
        
    for slide in SLIDE_PATHS:
        SLIDE_NAME = slide.split("/")[-1].split(".")[0]
        print("Startin on "+SLIDE_NAME)
        GroupName  = "Spot_Info"+"_"+SLIDE_NAME
        GroupInfo  = testana.create_group("/", GroupName)
        
        GroupName  = "F_Trajectory_Raw"+"_"+SLIDE_NAME
        GroupFraw  = testana.create_group("/", GroupName)
        
        GroupName  = "F_Trajectory_Fit"+"_"+SLIDE_NAME
        GroupFfit  = testana.create_group("/", GroupName)
        

        with tb.open_file(slide, mode="r", title="test") as tt:
            Num_Frames = tt.root.ImageStack._v_nchildren
            Frame_Name = []
            for kid in tt.root.ImageStack._v_children:
                Frame_Name.append(kid)

            mm = Image_Background(Frame_Name, Num_Frames, Shape, Frac)
            testana.create_array(GroupInfo, "Background", mm)
            rr = Summed_Image(Frame_Name, Num_Frames, Shape, mm)
            testana.create_array(GroupInfo, "Summed", mm)
            aa = Spot_finder(rr,10,3,Shape,20)
            testana.create_array(GroupInfo, "AllPairs", aa)

            aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)
            Num_Spots = len(aa)
            print("found "+str(Num_Spots)+" spots!")
            for x in range(0,Num_Spots):
                X = str(int(aa[x][0]))
                Y = str(int(aa[x][1]))
                NAME = "X"+X+"_Y"+Y
                sig = np.array(aaa[x::Num_Spots])[:,2]
                bkg = np.array(aaa[x::Num_Spots])[:,3]
                stuff = np.array([sig,bkg])
                testana.create_array(GroupFraw, NAME, stuff)
                
                SIG = sig
                BAK = bkg
                Num_Frames=500
                ww = np.arange(0,Num_Frames)*0.5
                sig = np.add.reduceat(SIG, np.arange(0, len(SIG), 5))
                bak = np.add.reduceat(BAK, np.arange(0, len(BAK), 5))
                www = np.add.reduceat(ww, np.arange(0, len(ww), 5))
                times    = np.arange(0,len(sig))*2.5
                baerry = np.sqrt(sig)
                naerry = np.sqrt(bak)
                coefsF = poly.polyfit(times,bak, 3)
                ffitF  = poly.polyval(times, coefsF)
                fitS = (sig-ffitF*25/56)/25
                fitSbk = baerry/25
                fitB = (bak-ffitF)/56
                fitBbk = naerry/25
                tim = times
                Meandiff, chiStep, LT = Step_fit(tim,fitS,fitSbk)
                chiLine     = Line_fit(tim,fitS,fitSbk)
                
                stuff = np.array([fitS, fitSbk, fitB, fitBbk, Meandiff, chiStep, chiLine, LT])
                testana.create_array(GroupFfit, NAME, stuff)
                
        #testana.flush()
    
    

In [ ]:
aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)

In [ ]:
tt.close()

In [ ]:
ANA_FILE = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana.h5"
data = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test.h5"
data = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/Slide1_Spot1.h5"
Shape = 221
Frac = 0.1
with tb.open_file(ANA_FILE, mode="w", title="test") as testana:
    GroupName  = "Spot_Info"
    GroupInfo  = testana.create_group("/", GroupName)
    GroupName  = "Signal_Area"
    GroupSig   = testana.create_group("/", GroupName)
    GroupName  = "Background_Area"
    GroupBkg   = testana.create_group("/", GroupName)
    
    with tb.open_file(data, mode="r", title="test") as tt:
        Num_Frames = tt.root.ImageStack._v_nchildren
        Frame_Name = []
        for kid in tt.root.ImageStack._v_children:
            Frame_Name.append(kid)
        
        mm = Image_Background(Frame_Name, Num_Frames, Shape, Frac)
        testana.create_array(GroupInfo, "Background", mm)
        rr = Summed_Image(Frame_Name, Num_Frames, Shape, mm)
        testana.create_array(GroupInfo, "Summed", mm)
        aa = Spot_finder(rr,9,3,Shape,10)
        testana.create_array(GroupInfo, "AllPairs", aa)
        
        aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)
        Num_Spots = len(aa)
        for x in range(0,Num_Spots):
            X = str(int(aa[x][0]))
            Y = str(int(aa[x][1]))
            NAME = "X"+X+"_Y"+Y
            sig = np.array(aaa[x::Num_Spots])[:,2]
            testana.create_array(GroupSig, NAME, sig)
            bkg = np.array(aaa[x::Num_Spots])[:,3]
            testana.create_array(GroupBkg, NAME, bkg)
        

In [ ]:
ttry = tb.open_file("/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana.h5"
                    , mode="r", title="test")

In [ ]:
ttry.root.Spot_Info.AllPairs

In [ ]:
ttry.close()

In [ ]:
data = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/Slide1_Spot1.h5"


In [ ]:
SLIDE_LOC = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/"

SLIDE_PATHS = [os.path.join(SLIDE_LOC, f) for f in os.listdir(SLIDE_LOC) if f.endswith(".h5")] 
SLIDE_NAMES = []
for slide in SLIDE_PATHS:
    hold = slide.split("/")[-1].split(".")[0]
    SLIDE_NAMES.append(hold)


In [ ]:
ANA_FILE = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana_full.h5"
#data = "/Users/austinmcdonald/Desktop/SMFI-Analysis/test.h5"
#data = "/Users/austinmcdonald/Desktop/SMFI-Analysis/10-12-17/Ba/Slide1_Spot1.h5"
Shape = 221
Frac = 0.1
    
with tb.open_file(ANA_FILE, mode="w", title="test") as testana:
        
    for slide in SLIDE_PATHS:
        SLIDE_NAME = slide.split("/")[-1].split(".")[0]
        print("Startin on "+SLIDE_NAME)
        GroupName  = "Spot_Info"+"_"+SLIDE_NAME
        GroupInfo  = testana.create_group("/", GroupName)
        GroupName  = "Signal_Area"+"_"+SLIDE_NAME
        GroupSig   = testana.create_group("/", GroupName)
        GroupName  = "Background_Area"+"_"+SLIDE_NAME
        GroupBkg   = testana.create_group("/", GroupName)

        with tb.open_file(slide, mode="r", title="test") as tt:
            Num_Frames = tt.root.ImageStack._v_nchildren
            Frame_Name = []
            for kid in tt.root.ImageStack._v_children:
                Frame_Name.append(kid)

            mm = Image_Background(Frame_Name, Num_Frames, Shape, Frac)
            testana.create_array(GroupInfo, "Background", mm)
            rr = Summed_Image(Frame_Name, Num_Frames, Shape, mm)
            testana.create_array(GroupInfo, "Summed", mm)
            aa = Spot_finder(rr,10,3,Shape,20)
            testana.create_array(GroupInfo, "AllPairs", aa)

            aaa = Spot_Area(Frame_Name, Num_Frames, mm, aa)
            Num_Spots = len(aa)
            print("found "+str(Num_Spots)+" spots!")
            for x in range(0,Num_Spots):
                X = str(int(aa[x][0]))
                Y = str(int(aa[x][1]))
                NAME = "X"+X+"_Y"+Y
                sig = np.array(aaa[x::Num_Spots])[:,2]
                testana.create_array(GroupSig, NAME, sig)
                bkg = np.array(aaa[x::Num_Spots])[:,3]
                testana.create_array(GroupBkg, NAME, bkg)
        #testana.flush()
    
    

In [ ]:
ttry = tb.open_file("/Users/austinmcdonald/Desktop/SMFI-Analysis/test_ana_full.h5"
                    , mode="r", title="test")

In [ ]:
ttry.root.Spot_Info_Slide1_Spot1

In [ ]:
ttry.close()

In [ ]:
np.array(aaa[9::len(aa)])

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
qq = np.array(aaa[21::len(aa)])
ww = np.arange(0,Num_Frames)*0.5
plt.plot(ww,qq[:,2]/25,color='r')
plt.plot(ww,qq[:,3]/56,color='k')
plt.show()
aaaa = running_mean(qq[:,2]/25,  5)
aaaaB = running_mean(qq[:,3]/56,  5)
wwww = running_mean(ww, 5)

plt.plot(wwww,aaaa,color='r')
plt.plot(wwww,aaaaB,color='k')

#valsF =np.arange(0,len(dataFull))*2.5
coefsF = poly.polyfit(wwww,aaaaB, 9)
ffitF  = poly.polyval(wwww, coefsF)
plt.scatter(wwww,ffitF)
plt.show()

ffitF  = poly.polyval(wwww, coefsF)
plt.plot(wwww,aaaa-ffitF,color='r')
plt.plot(wwww,aaaaB-ffitF,color='k')

plt.show()

In [ ]:
tt.close()

In [ ]:
#################################################################
#  FirstImage Should be an image from the stack of images, it is easiest to choose the first
#  eCoeff/eOffset are the units given in the tiff file that allow you to go from the "grey count" 
#  to acutual Photo Electrons PE. These can be found by opening a tiff with a text editing sofware and 
#  scrolling to the bottom.
#
#  The puropose of this function is to get the shape of the image and generate matricies of variables 
#  for later use
#################################################################
def Shaper(FirstImage,eCoeff,eOffset):
    ShapeSize = np.array(Image.open(FirstImage))
    Shape = ShapeSize.shape[0]
    eCoeffMatrix = eCoeff*np.ones(Shape**2).reshape((Shape,Shape))
    eOffsetMatrix = eOffset*np.ones(Shape**2).reshape((Shape,Shape))
    EmptyData = np.zeros(Shape**2).reshape((Shape,Shape))
    return Shape, EmptyData, eCoeffMatrix, eOffsetMatrix

#################################################################
# datafiles is the list of images in the stack ( should be in order)
# Shape is the shape of the image 
# eCoeffMatrix / eOffsetMatrix are generated from Shaper above
# Frac is the percent of the images at the end of the stack you want to 
# use to generate the background profile 
#################################################################
def Image_Background(datafiles, Shape, eCoeffMatrix, eOffsetMatrix, Frac):
    xvs = np.arange(0,Shape)
    MeanFit = np.zeros(Shape**2).reshape((Shape,Shape))
    LL = len(datafiles)
    Fraction = int(LL*Frac)
    for x in range(LL-Fraction,LL):
        Images = np.array(Image.open(datafiles[x]))
        Images = eCoeffMatrix*(Images - eOffsetMatrix)  
        ImageFit = []
        for y in range(0,Shape):
            coefs = poly.polyfit(xvs,Images[y], 6) 
            ffit  = poly.polyval(xvs, coefs)
            ImageFit.append(ffit)
        MeanFit+=np.array(ImageFit)
    MeanFit = MeanFit/(Fraction)
    return MeanFit

#################################################################
# datafiles is the list of images in the stack ( should be in order)
# EmptyData is the empty matrix generated by Shaper 
# eCoeffMatrix / eOffsetMatrix are generated from Shaper above
# MeanFit is generated above 
# 
# This returns the summed stack where each image is background corrected 
# this is needed to find the high points with the next function
#################################################################

def Summed_Image(datafiles, EmptyData, MeanFit, eCoeffMatrix, eOffsetMatrix):
    LL = len(datafiles)
    for q in range(0,LL):
        Images = (np.array(Image.open(datafiles[q])) )
        Images = eCoeffMatrix*(Images - eOffsetMatrix)-MeanFit 
        EmptyData += Images
    return EmptyData

#################################################################
# datafiles is the list of images in the stack ( should be in order)
# This function scans the data and finds maxima in a region that is the size of 
# neighborhood_size. It looks for points in the neighborhood_size that are above
# the threshold. The threshold is defined as the mean of the data set + threshold_sigma
# deavations above the mean. the lower the threshold_sigma the more points you will get.
# also lowering the neighborhood_size will yeild more points
#
# Shape and EdgeCut are included so you can cut points out that are near the edge of the image
#################################################################
def Spot_finder(SummedData, neighborhood_size, threshold_sigma, Shape, EdgeCut):

    threshold = np.mean(SummedData)+threshold_sigma*np.std(SummedData)

    data_max = filters.maximum_filter(SummedData, neighborhood_size)
    maxima = (SummedData == data_max)
    data_min = filters.minimum_filter(SummedData, neighborhood_size)
    diff = ((data_max - data_min) > threshold)
    maxima[diff == 0] = 0

    labeled, num_objects = ndimage.label(maxima)
    slices = ndimage.find_objects(labeled)
    x, y = [], []
    AllPairs = []
    for dy,dx in slices:
        x_center = (dx.start + dx.stop - 1)/2
        y_center = (dy.start + dy.stop - 1)/2
        if x_center>EdgeCut and x_center<Shape-EdgeCut and y_center>EdgeCut and y_center<Shape-EdgeCut:
            x.append(x_center)
            y.append(y_center)
            AllPairs.append((x_center,y_center))
    return AllPairs, x, y


In [ ]:
%%time
eCoeff = 0.01160
eOffset= 1917

In [ ]:
eCoeff = 0.01160
eOffset= 1917

Shape, EmptyData, eCoeffMatrix, eOffsetMatrix = Shaper(files[0],eCoeff,eOffset)

MeanFit = Image_Background(files, Shape, eCoeffMatrix, eOffsetMatrix, 0.05)

Data = Summed_Image(files, EmptyData, MeanFit, eCoeffMatrix, eOffsetMatrix)
#Data[Data < 0] = 0

All,x_points,y_points =Spot_finder(Data, 20, 6, Shape, 20)

len(All)

In [ ]:
print('All ',len(All))
print('x ',len(x_points))
print('y ',len(y_points))

In [ ]:
All

In [ ]:
plt.imshow(Data)
plt.show()

plt.imshow(Data)
plt.scatter(x_points,y_points,color='r')
plt.show()

Xindex = int(All[13][0])
Yindex = int(All[13][1])
print(All[13])
Row = int(4) 
Col = int(4)
spot1 = np.array(Data[Yindex-Row:Yindex+Row+1,Xindex-Col:Xindex+Col+1])

plt.imshow(spot1)
plt.show()

spot = spot1[2:7,2:7]
plt.imshow(spot)
plt.show()

In [ ]:
#################################################################
# Takes the datafiles and the IDed points and computes the area for each point
# through the whole stack
#################################################################
def Spot_Area(datafiles, MeanFit, eCoeffMatrix, eOffsetMatrix, All):
    LL = len(datafiles)
    SpotInfo = []
    for q in range(0,LL):
        Images = (np.array(Image.open(datafiles[q])) )
        Images = eCoeffMatrix*(Images - eOffsetMatrix)-MeanFit 
        Images[Images < 0] = 0 

        Row = int(4) # define the area to take around the spots
        Col = int(4) # define the area to take around the spots
        for we in range(0,len(All)):
            Xindex = int(All[we][0])
            Yindex = int(All[we][1])

            spot1 = np.array(Images[Yindex-Row:Yindex+Row+1,Xindex-Col:Xindex+Col+1])
            spot2 = np.array(Images[Yindex-Row:Yindex+Row+1,Xindex-Col:Xindex+Col+1])
            spot = spot1[2:7,2:7]
            SpotRemove = np.zeros(5**2).reshape((5,5))
            spot2[2:7,2:7]=SpotRemove
            spotB = spot2

            ATot = spot.sum()
            ATotB = spot2.sum()
            SpotInfo.append([Xindex,Yindex,ATot,ATotB])
    return SpotInfo

In [ ]:
len(files)

In [ ]:
AHH = Spot_Area(files, MeanFit, eCoeffMatrix, eOffsetMatrix, All)

In [ ]:
HoldArea = np.array(AHH[2::36])[:,2]
#HoldAreaB = np.array(SpotInfo2[x::Par])[:,5]

In [ ]:
len(HoldArea)

In [ ]:
Par = len(AllPairs)
AVG = 5
Signal = []
BackG  = []
SignalF = []
BackGF  = []
for x in range(0,Par):
    HoldArea = np.array(SpotInfo2[x::Par])[:,4]
    HoldAreaB = np.array(SpotInfo2[x::Par])[:,5]
    aa = np.add.reduceat(HoldArea, np.arange(0, len(HoldArea), AVG))
    aaB = np.add.reduceat(HoldAreaB, np.arange(0, len(HoldAreaB), AVG))
    vals =np.arange(0,len(aa))

    coefs = poly.polyfit(vals,aaB, 3)
    ffit  = poly.polyval(vals, coefs)
    Sig = np.array(aa-ffit)
    bac = np.array(aaB-ffit)

    SigF = np.array(HoldArea)
    bacF = np.array(HoldAreaB)

    Signal.append(Sig)
    BackG.append(bac)
    SignalF.append(SigF)
    BackGF.append(bacF)

## 

In [ ]:
AVG = 5
Mnumb = 5
M = 36

HoldArea = np.array(AHH[M::36])[:,2]
HoldAreaB = np.array(AHH[M::36])[:,3]
aa = np.add.reduceat(HoldArea, np.arange(0, len(HoldArea), AVG))
aaB = np.add.reduceat(HoldAreaB, np.arange(0, len(HoldAreaB), AVG))
vals =np.arange(0,len(aa))

coefs = poly.polyfit(vals,aaB, 3)
ffit  = poly.polyval(vals, coefs)
Sig = np.array(aa-ffit*(25/56))
Bak = np.array(aaB-ffit)

xx = np.arange(500)
M = 25
plt.scatter(vals,Sig,color='r')
plt.scatter(vals,Bak,color='k')

plt.show()

In [ ]:
len(np.arange(500))

In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
#Raw plot
M = 13

HoldArea = np.array(AHH[M::36])[:,2]
HoldAreaB = np.array(AHH[M::36])[:,3]

dataFull = np.add.reduceat(HoldArea, np.arange(0, len(HoldArea), 5))
dataFullB = np.add.reduceat(HoldAreaB, np.arange(0, len(HoldAreaB), 5))
times    = np.arange(0,len(dataFull))*2.5

K=5
Atime = running_mean(times,K)
Adat  = running_mean(dataFull,K)
AdatB = running_mean(dataFullB,K)

fig, ax = plt.subplots(figsize=(9,9))

b1 = ax.scatter(times-5,dataFullB, color='k',marker='s',label='Background', alpha=0.5,s=75)
bf1 = ax.plot(Atime-5,AdatB,color='k',label='moving avg',linestyle=':',linewidth=4)

s1 = ax.scatter(times-5,dataFull,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
sf1 = ax.plot(Atime-5,Adat,color='firebrick',label='moving avg',linestyle='-',linewidth=4)

valsF =np.arange(0,len(dataFull))*2.5
coefsF = poly.polyfit(valsF,dataFullB, 9)
ffitF  = poly.polyval(valsF, coefsF)
plt.scatter(valsF,ffitF)

baerry = np.sqrt(dataFull)
naerry = np.sqrt(dataFullB)
ax.errorbar(times-5,dataFull,yerr=baerry,color='firebrick',fmt='.')
ax.errorbar(times-5,dataFullB,yerr=naerry,color='k',fmt='.')

ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()


#Subtracted plot
fig, ax = plt.subplots(figsize=(9,9))

dataSub  = dataFull-ffitF*(25/56)
dataSubB = dataFullB-ffitF
Adat  = running_mean(dataSub,K)
AdatB = running_mean(dataSubB,K)

ax.scatter(times-5,dataSubB, color='k',marker='s',label='Background', alpha=0.5,s=75)
ax.plot(Atime-5,AdatB,color='k',label='moving avg',linestyle=':',linewidth=4)

ax.scatter(times-5,dataSub,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
ax.plot(Atime-5,Adat,color='firebrick',label='moving avg',linestyle='-',linewidth=4)
ax.errorbar(times-5,dataSub,yerr=baerry,color='firebrick',fmt='.')
ax.errorbar(times-5,dataSubB,yerr=naerry,color='k',fmt='.')



ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()


#per pixel plot

fig, ax = plt.subplots(figsize=(9,9))

bknorm=56
signorm=25

ax.scatter(times-5,dataSubB/bknorm, color='k',marker='s',label='Background', alpha=0.5,s=75)
ax.plot(Atime-5,AdatB/bknorm,color='k',label='moving avg',linestyle=':',linewidth=4)

ax.scatter(times-5,dataSub/signorm,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
ax.plot(Atime-5,Adat/signorm,color='firebrick',label='moving avg',linestyle='-',linewidth=4)
ax.errorbar(times-5,dataSub/signorm,yerr=baerry/signorm,color='firebrick',fmt='.')
ax.errorbar(times-5,dataSubB/bknorm,yerr=naerry/bknorm,color='k',fmt='.')




ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Raw plot
M = 21

HoldArea = np.array(AHH[M::36])[:,2]
HoldAreaB = np.array(AHH[M::36])[:,3]

dataFull = np.add.reduceat(HoldArea, np.arange(0, len(HoldArea), 5))
dataFullB = np.add.reduceat(HoldAreaB, np.arange(0, len(HoldAreaB), 5))
times    = np.arange(0,len(dataFull))*2.5

K=5
Atime = running_mean(times,K)
Adat  = running_mean(dataFull,K)
AdatB = running_mean(dataFullB,K)

fig, ax = plt.subplots(figsize=(9,9))

b1 = ax.scatter(times-5,dataFullB, color='k',marker='s',label='Background', alpha=0.5,s=75)
bf1 = ax.plot(Atime-5,AdatB,color='k',label='moving avg',linestyle=':',linewidth=4)

s1 = ax.scatter(times-5,dataFull,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
sf1 = ax.plot(Atime-5,Adat,color='firebrick',label='moving avg',linestyle='-',linewidth=4)

valsF =np.arange(0,len(dataFull))*2.5
coefsF = poly.polyfit(valsF,dataFullB, 9)
ffitF  = poly.polyval(valsF, coefsF)
plt.scatter(valsF,ffitF)

baerry = np.sqrt(dataFull)
naerry = np.sqrt(dataFullB)
ax.errorbar(times-5,dataFull,yerr=baerry,color='firebrick',fmt='.')
ax.errorbar(times-5,dataFullB,yerr=naerry,color='k',fmt='.')

ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()


#Subtracted plot
fig, ax = plt.subplots(figsize=(9,9))

dataSub  = dataFull-ffitF*(25/56)
dataSubB = dataFullB-ffitF
Adat  = running_mean(dataSub,K)
AdatB = running_mean(dataSubB,K)

ax.scatter(times-5,dataSubB, color='k',marker='s',label='Background', alpha=0.5,s=75)
ax.plot(Atime-5,AdatB,color='k',label='moving avg',linestyle=':',linewidth=4)

ax.scatter(times-5,dataSub,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
ax.plot(Atime-5,Adat,color='firebrick',label='moving avg',linestyle='-',linewidth=4)
ax.errorbar(times-5,dataSub,yerr=baerry,color='firebrick',fmt='.')
ax.errorbar(times-5,dataSubB,yerr=naerry,color='k',fmt='.')



ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()


#per pixel plot

fig, ax = plt.subplots(figsize=(9,9))

bknorm=56
signorm=25

ax.scatter(times-5,dataSubB/bknorm, color='k',marker='s',label='Background', alpha=0.5,s=75)
ax.plot(Atime-5,AdatB/bknorm,color='k',label='moving avg',linestyle=':',linewidth=4)

ax.scatter(times-5,dataSub/signorm,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
ax.plot(Atime-5,Adat/signorm,color='firebrick',label='moving avg',linestyle='-',linewidth=4)
ax.errorbar(times-5,dataSub/signorm,yerr=baerry/signorm,color='firebrick',fmt='.')
ax.errorbar(times-5,dataSubB/bknorm,yerr=naerry/bknorm,color='k',fmt='.')




ax.set_xlim(0,375)
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Raw plot
M = 21

HoldArea = np.array(AHH[M::36])[:,2]
HoldAreaB = np.array(AHH[M::36])[:,3]

dataFull = np.add.reduceat(HoldArea, np.arange(0, len(HoldArea), 5))
dataFullB = np.add.reduceat(HoldAreaB, np.arange(0, len(HoldAreaB), 5))
times    = np.arange(0,len(dataFull))*2.5

#Atime = running_mean(times,K)
#Adat  = running_mean(dataFull,K)
#AdatB = running_mean(dataFullB,K)

fig, ax = plt.subplots(figsize=(9,9))



valsF =np.arange(0,len(dataFull))*2.5
coefsF = poly.polyfit(valsF,dataFullB, 9)
ffitF  = poly.polyval(valsF, coefsF)

baerry = np.sqrt(dataFull)
naerry = np.sqrt(dataFullB)


dataSub  = dataFull-ffitF*(25/56)
dataSubB = dataFullB-ffitF
K=5
Adat  = running_mean(dataSub,K)
AdatB = running_mean(dataSubB,K)



bknorm=56
signorm=25

ax.scatter(times-5,dataSubB/bknorm, color='k',marker='s',label='Background', alpha=0.5,s=75)
ax.plot(Atime-5,AdatB/bknorm,color='k',label='moving avg',linestyle=':',linewidth=4)

ax.scatter(times-5,dataSub/signorm,color='firebrick',marker='^',label='Signal', alpha=0.5,s=100)
ax.plot(Atime-5,Adat/signorm,color='firebrick',label='moving avg',linestyle='-',linewidth=4)
ax.errorbar(times-5,dataSub/signorm,yerr=baerry/signorm,color='firebrick',fmt='.')
ax.errorbar(times-5,dataSubB/bknorm,yerr=naerry/bknorm,color='k',fmt='.')




ax.set_xlim(0,max(times))
#ax.set_ylim(0,15)
ax.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
Shape

In [ ]:
Xc=[]
Yc=[]
cut = 20
for qw in range(0,len(AllPairs)):
    XX = AllPairs[qw][0]
    YY = AllPairs[qw][1]
    if XX>cut and XX<Shape-cut and YY>cut and YY<Shape-cut:
        Xc.append(XX)
        Yc.append(YY)
len(Xc)

In [ ]:
Pairs[0]

In [ ]:
eCoeff = 0.01160
eOffset= 1917

ShapeSize = np.array(Image.open(files[0]))
Shape = ShapeSize.shape[0]
eCoeffM = eCoeff*np.ones(Shape**2).reshape((Shape,Shape))
eOffsetM = eOffset*np.ones(Shape**2).reshape((Shape,Shape))
DataSub = np.zeros(Shape**2).reshape((Shape,Shape))

neighborhood_size = 20

AllPairs =[]

xvs = np.arange(0,Shape)
MeanFit = np.zeros(Shape**2).reshape((Shape,Shape))
for x in range(190,200):
    Images = np.array(Image.open(files[x]))
    Images = eCoeffM*(Images - eOffsetM) ######################################## 
    ImageFit = []
    for y in range(0,Shape):
        coefs = poly.polyfit(xvs,Images[y], 6) # fits 11th order poly 
        ffit  = poly.polyval(xvs, coefs)
        ImageFit.append(ffit)
    MeanFit+=np.array(ImageFit)
MeanFit = MeanFit/10
Images=None

for q in range(0,200):
    Images = (np.array(Image.open(files[q])) )###################### - MeanFit
    Images = eCoeffM*(Images - eOffsetM)#-MeanFit      ######################################## 
    DataSub += Images


data = DataSub

threshold = np.mean(data)+6*np.std(data)

data_max = filters.maximum_filter(data, neighborhood_size)
maxima = (data == data_max)
data_min = filters.minimum_filter(data, neighborhood_size)
diff = ((data_max - data_min) > threshold)
maxima[diff == 0] = 0

labeled, num_objects = ndimage.label(maxima)
slices = ndimage.find_objects(labeled)
x, y = [], []
Pairs = []
for dy,dx in slices:
    x_center = (dx.start + dx.stop - 1)/2
    x.append(x_center)
    y_center = (dy.start + dy.stop - 1)/2    
    y.append(y_center)
    Pairs.append((x_center,y_center))
for item in Pairs:
    if item not in AllPairs:
        AllPairs.append(item) 

In [ ]:
len(AllPairs)

In [ ]:
AllPairs

In [ ]:
np.mean(DataSub)


In [ ]:
DataSub

In [ ]:
MeanFit*200

In [ ]:
np.mean(DataSub-MeanFit*200)

In [ ]:
'''Xc=[]
Yc=[]
for qw in range(0,len(AllPairs)):
    XX = AllPairs[qw][0]
    YY = AllPairs[qw][1]
    if XX>10 and XX<210 and YY>10 and YY<210:
        Xc.append(XX)
        Yc.append(YY)'''
Xc=[]
Yc=[]
for qw in range(0,len(AllPairs)):
    XX = AllPairs[qw][0]
    YY = AllPairs[qw][1]
    Xc.append(XX)
    Yc.append(YY)

In [ ]:
dd=np.array(DataSub)
plt.imshow(dd)
plt.show()
plt.imshow(dd)
plt.scatter(Xc,Yc,color='r')
plt.show()

In [ ]:
dd=np.array(DataSub-MeanFit*200)
plt.imshow(dd)
plt.show()

In [ ]:
tt = np.array(MeanFit)
plt.imshow(tt)
plt.show()